In [1]:
import json
from rank_bm25 import BM25Okapi
import numpy as np

In [2]:
# Pour chaquer fichier (dev, test, train), on crée un dictionnaire ayant pr clés question id, doc id...

def data(type): # type peut être dev, test ou train.

    # Define the path to your text document
    file_path = rf'WikiPassageQA\{type}.txt'
    # Initialize an empty dictionary to hold your data
    data_dict = {}
    articles_with_ids = {}

    # Open the text document for reading
    with open(file_path, 'r') as file:
            # Skip the header line
            next(file)
            # Iterate over each line in the file
            for line in file:
                # Split the line into components based on tabs
                parts = line.strip().split('\t')
                # Extract the individual components
                qid, question, doc_id, doc_name, rel_passages = parts
                # Convert the QID to an integer (if you want it as an integer)
                qid = int(qid)
                # Convert DocumentID to an integer (if needed)
                doc_id = int(doc_id)
                # Split 'RelevantPassages' into a list of integers (if they are always numbers)
                rel_passages = [int(x) for x in rel_passages.split(',')]
                # Populate the dictionary
                data_dict[qid] = {
                    'Question': question,
                    'DocumentID': doc_id,
                    'DocumentName': doc_name,
                    'RelevantPassages': rel_passages
                }
                articles_with_ids[doc_id] = doc_name[:-5]
        
    return data_dict, articles_with_ids

# At this point, data_dict contains your data structured as required
# articles_with_ids : dict mapping document IDs to their Wikipedia article titles

data_dict, _ = data("test")

In [3]:
def load_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data

In [4]:
print(data_dict.keys())
print(data_dict[449])

dict_keys([449, 1140, 782, 1421, 3738, 844, 1201, 4076, 404, 3915, 3675, 104, 1214, 1402, 3551, 3981, 78, 1890, 1849, 244, 3039, 256, 4167, 3741, 836, 3853, 1836, 4184, 1871, 3185, 3451, 1737, 332, 1762, 1147, 2293, 3149, 282, 1378, 1088, 3702, 2848, 224, 2345, 2261, 3316, 3654, 2348, 3719, 3503, 1507, 2056, 2911, 2704, 3992, 4029, 2359, 3872, 2236, 1686, 1117, 1222, 2908, 1366, 1407, 3801, 1321, 1498, 1898, 718, 2393, 1033, 2620, 2524, 3931, 1135, 1020, 2536, 1781, 1572, 515, 601, 652, 1053, 136, 1162, 1023, 2770, 4, 3010, 762, 178, 651, 323, 3593, 1061, 2371, 1331, 1703, 101, 2533, 139, 642, 3865, 3232, 3245, 3869, 462, 3490, 345, 1722, 494, 3210, 631, 820, 1535, 252, 1150, 3327, 3822, 2023, 287, 3928, 1311, 2708, 1954, 3286, 493, 1000, 2547, 659, 983, 6, 3223, 2360, 1869, 1756, 1794, 2508, 3538, 3125, 2321, 341, 3754, 723, 2914, 174, 29, 2821, 1609, 935, 994, 76, 2052, 1383, 4100, 1996, 1259, 3199, 2312, 60, 1571, 1111, 3227, 3421, 2158, 657, 3923, 2421, 2677, 3623, 685, 3445, 3256,

In [5]:
filepath = 'WikiPassageQA/document_passages_tokenized.json'
doc_passages = load_data(filepath)

print(doc_passages['543'])

{'56': ['iraq', 'intends', 'increase', 'production', 'million', 'barrel', 'per', 'day', 'oil', 'well', 'drilled', 'iraq', 'compared', 'million', 'well', 'texas', 'alone', 'iraq', 'one', 'founding', 'member', 'opec', 'despite', 'improved', 'security', 'billion', 'dollar', 'oil', 'revenue', 'iraq', 'still', 'generates', 'half', 'electricity', 'customer', 'demand', 'leading', 'protest', 'hot', 'summer', 'month', 'iraq', 'oil', 'law', 'proposed', 'piece', 'legislation', 'submitted', 'iraqi', 'council', 'representative', 'may', 'iraqi', 'government', 'yet', 'reach', 'agreement', 'law'], '77': ['however', 'medieval', 'era', 'baghdad', 'capital', 'abbasid', 'caliphate', 'iraqi', 'kitchen', 'reached', 'zenith', 'today', 'cuisine', 'iraq', 'reflects', 'rich', 'inheritance', 'well', 'strong', 'influence', 'culinary', 'tradition', 'neighbouring', 'turkey', 'iran', 'greater', 'syria', 'area', 'characteristic', 'ingredient', 'iraqi', 'cuisine', 'include', 'vegetable', 'aubergine', 'tomato', 'okra',

In [6]:
def shuffle_list(L1, L2):
    # Créer une liste de la même taille que L1, initialisée avec None ou une valeur neutre
    result = [None] * len(L1)
    
    # Assigner chaque élément de L1 à la nouvelle position indiquée par L2
    for original_index, new_index in enumerate(L2):
        result[new_index] = L1[original_index]
    
    return result

Premier test de BM25 directement sur document_passages_tokenized_2.json (donc de la triche, car il faut tester sur notre dictionnaire acquis via scraping (articles wiki décomposés en passages de 6 sentences))

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Initialisation de la liste de stop words et du lemmatiseur
stop_words = set(stopwords.words('english'))
# stop_words = set() # Je teste sans tej les "the, if..."
lemmatizer = WordNetLemmatizer()


def fetch_passages(doc_id, doc_passages):
    # Récupérer les passages d'un document spécifique
    passages = [doc_passages[doc_id][pid] for pid in doc_passages[doc_id]]
    pid = [int(pid) for pid in doc_passages[doc_id]]
    passages = shuffle_list(passages,pid)
    return passages

def rank_passages(query, passages):
    tokens = word_tokenize(query)
    # Suppression des mots vides et nettoyage des caractères non alphabétiques
    tokens = [word.lower() for word in tokens if word.lower() not in stop_words and word.isalpha()]
    # Lemmatisation
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    tokenized_query = tokens

    # Créer l'objet BM25 avec les passages tokenisés
    bm25 = BM25Okapi(passages)

    # Calculer les scores pour la requête
    scores = bm25.get_scores(tokenized_query)
    
    # Trier les passages par score en ordre décroissant
    ranked_passages = np.argsort(scores)[::-1]
    
    return ranked_passages, scores

# Supposons que nous traitons la question ID 0 dans data_dict
question_data = data_dict[449]
query = question_data['Question']
doc_id = str(question_data['DocumentID'])  # Assurez-vous que l'ID est en format string si nécessaire

# Récupérer les passages du document spécifique
passages = fetch_passages(doc_id, doc_passages)

# Classement des passages pour la requête
ranked_passages, scores = rank_passages(query, passages)

# Affichage des résultats
for idx in ranked_passages:
    print(f"Passage ID: {idx}, Score: {scores[idx]}, Text: {' '.join(passages[idx])}")


Passage ID: 62, Score: 5.774784482009735, Text: iraq population kurd assyrian turkmen much smaller minority mandean armenian circassian iranian shabakis yazidis kawliya make remainder population around marsh arab live southern iraq iraq community chechen southern iraq community iraqi african descent legacy slavery practised islamic caliphate beginning zanj rebellion century basra role key port populous country arabian plate
Passage ID: 42, Score: 5.152851602025532, Text: party consistent perspective rival kurdistan list dominated two party kurdistan democratic party led masood barzani patriotic union kurdistan headed jalal talabani party secular enjoy close tie west according failed state index iraq world seventh politically unstable country concentration power hand prime minister nouri growing pressure opposition led growing concern future political right iraq nevertheless progress made country risen place
Passage ID: 46, Score: 4.638126620134543, Text: declaration enraged many local 

Ca marche pas mal, ici les relevant passages étaient 42 et 43.

Passons maintenant au réel test, avec comme critère la correspondance par bigramme 

In [8]:
from nltk import bigrams
from collections import Counter

def calculate_bigram_overlap(doc1, doc2):
    # Générer des bigrammes pour chaque document
    bigrams1 = list(bigrams(doc1))
    bigrams2 = list(bigrams(doc2))
    
    # Compter les bigrammes dans chaque document
    bigram_counts1 = Counter(bigrams1)
    bigram_counts2 = Counter(bigrams2)
    
    # Trouver les bigrammes communs et calculer le nombre total de bigrammes partagés
    common_bigrams = bigram_counts1 & bigram_counts2
    total_shared = sum(common_bigrams.values())
    
    # Calculer le nombre total de bigrammes dans le plus petit document (pour calculer le pourcentage de recouvrement)
    total_bigrams = min(len(bigrams1), len(bigrams2))
    
    # Calculer le pourcentage de recouvrement
    if total_bigrams > 0:
        overlap_percentage = (total_shared / total_bigrams) * 100
    else:
        overlap_percentage = 0
    
    # Retourner 1 si le pourcentage est supérieur à 15%, sinon 0
    return 1 if overlap_percentage > 15 else 0


In [9]:
filepath = './data_tokenized.json'
wiki_docs = load_data(filepath)

In [10]:
for doc in wiki_docs:
    new_L = []
    for L in wiki_docs[doc]:
        if len(L)>5:
            new_L.append(L)
    wiki_docs[doc] = new_L

In [11]:
print(wiki_docs.keys())
print(wiki_docs["543"])

dict_keys(['672', '359', '285', '579', '204', '2', '430', '341', '420', '561', '765', '189', '817', '328', '97', '742', '603', '788', '6', '135', '512', '263', '462', '598', '348', '478', '547', '707', '59', '353', '600', '757', '480', '700', '206', '807', '852', '23', '310', '612', '588', '526', '734', '824', '237', '283', '559', '804', '661', '544', '558', '479', '79', '465', '110', '311', '516', '724', '426', '491', '616', '781', '56', '266', '175', '472', '585', '314', '744', '562', '844', '254', '317', '299', '521', '775', '257', '64', '610', '1', '680', '433', '840', '15', '417', '260', '854', '255', '514', '120', '129', '365', '413', '33', '436', '779', '477', '640', '719', '198', '344', '519', '693', '9', '119', '381', '522', '557', '468', '67', '473', '753', '0', '784', '760', '117', '535', '48', '74', '225', '83', '727', '704', '139', '130', '52', '801', '167', '714', '568', '185', '42', '17', '318', '12', '684', '660', '584', '30', '628', '137', '454', '252', '31', '8', '155

In [12]:
# Supposons que nous traitons la question ID 449 dans data_dict
question_data = data_dict[449]
query = question_data['Question']
doc_id = str(question_data['DocumentID'])  # Assurez-vous que l'ID est en format string si nécessaire
pertinents = question_data['RelevantPassages']

passages = wiki_docs[doc_id]
# Classement des passages pour la requête
ranked_passages, scores = rank_passages(query, passages)

# Affichage des résultats
for idx in ranked_passages:
    per = 0 # per is equal to 1 if the considered passage is deemed relevant, 0 otherwise
    for rel_id in pertinents: # rel_id stands for relevant id 
        if calculate_bigram_overlap(passages[idx],doc_passages[doc_id][str(rel_id)]): # bigram overlapping as a metric of pertinence 
            per = 1
    print(f"Passage ID: {idx}, Pertinent: {per}, Score: {scores[idx]}, Text: {' '.join(passages[idx])}")

Passage ID: 54, Pertinent: 0, Score: 4.720581698358184, Text: year personal status law made polygamy extremely difficult granted child custody mother case divorce prohibited repudiation marriage age article civil code also identifies islamic law formal source law iraq sharia court civil court used sharia issue personal status including marriage divorce iraq introduced sharia punishment certain type criminal offence code based french civil law well sunni jafari interpretation cpa chief executive paul bremer said would veto constitutional draft stating sharia principal basis law declaration enraged many local shia cleric united state relented allowing role sharia constitution help end stalemate draft constitution
Passage ID: 44, Pertinent: 0, Score: 4.154661979235281, Text: december dozen iraqi shia militiaman supporter marched green zone baghdad surrounded embassy three day later amid rising tension united state iran launched drone strike convoy traveling near baghdad airport killing qa

In [13]:
print(data_dict[449])
print(doc_passages["543"]['43'])

{'Question': "What is Iraq's role in political unstabilization?", 'DocumentID': 543, 'DocumentName': 'Iraq.html', 'RelevantPassages': [42, 43]}
['august', 'reign', 'came', 'end', 'announced', 'august', 'would', 'stand', 'aside', 'haider', 'nominated', 'day', 'earlier', 'newly', 'installed', 'president', 'fuad', 'masum', 'could', 'take', 'point', 'clung', 'power', 'even', 'asking', 'federal', 'court', 'veto', 'president', 'nomination', 'describing', 'violation', 'constitution', 'transparency', 'international', 'rank', 'iraq', 'government', 'government', 'world', 'government', 'payroll', 'increased', 'million', 'employee', 'saddam', 'hussein', 'around', 'million', 'employee', 'combination', 'decreased', 'oil', 'price', 'government', 'budget', 'deficit', 'near', 'gdp']


In [14]:
def evaluate_performance(data_dict, wiki_docs):
    all_ap = []
    all_p_at_5 = []
    all_recall_at_5 = []
    all_p_at_10 = []
    all_recall_at_10 = []
    all_recall_at_20 = []
    all_mrr = []
    all_ndcg = []

    # Itérer sur chaque question dans data_dict
    for q_id, q_data in data_dict.items():
        query = q_data['Question']
        doc_id = str(q_data['DocumentID'])
        relevant_passages = q_data['RelevantPassages']
        
        # Obtenir les passages du document
        # passages = wiki_docs[doc_id]
        passages = fetch_passages(doc_id, wiki_docs)

        
        # Classement des passages pour la requête
        ranked_passages_index, scores = rank_passages(query, passages)
        
        # Identifier les passages pertinents
        is_relevant = [0] * len(passages)
        for idx, ranked_idx in enumerate(ranked_passages_index):
            for rel_id in relevant_passages:
                if calculate_bigram_overlap(passages[ranked_idx], doc_passages[doc_id][str(rel_id)]):
                    is_relevant[idx] = 1
                    break

        # Calculer les métriques pour cette requête
        ap = calculate_average_precision(is_relevant)
        p_at_5 = calculate_precision_at_k(is_relevant, 5)
        recall_at_5 = calculate_recall_at_k(is_relevant, 5, len(relevant_passages))
        p_at_10 = calculate_precision_at_k(is_relevant, 10)
        recall_at_10 = calculate_recall_at_k(is_relevant, 10, len(relevant_passages))
        recall_at_20 = calculate_recall_at_k(is_relevant, 20, len(relevant_passages))

        all_ap.append(ap)
        all_p_at_5.append(p_at_5)
        all_recall_at_5.append(recall_at_5)
        all_p_at_10.append(p_at_10)
        all_recall_at_10.append(recall_at_10)
        all_recall_at_20.append(recall_at_20)

        # Calcul des métriques MRR et nDCG
        all_mrr.append(calculate_mrr(is_relevant))

        ideal_relevant = sorted(is_relevant, reverse=True)  # Idéal pour nDCG
        all_ndcg.append(calculate_ndcg(is_relevant, ideal_relevant))

    # Calculer les moyennes pour toutes les requêtes
    MAP = sum(all_ap) / len(all_ap)
    mean_p_at_5 = sum(all_p_at_5) / len(all_p_at_5)
    mean_recall_at_5 = sum(all_recall_at_5) / len(all_recall_at_5)
    mean_p_at_10 = sum(all_p_at_10) / len(all_p_at_10)
    mean_recall_at_10 = sum(all_recall_at_10) / len(all_recall_at_10)
    mean_recall_at_20 = sum(all_recall_at_20) / len(all_recall_at_20)
    mean_mrr = np.mean(all_mrr)
    mean_ndcg = np.mean(all_ndcg)

    return MAP, mean_p_at_5, mean_recall_at_5, mean_p_at_10, mean_recall_at_10, mean_recall_at_20, mean_mrr, mean_ndcg

# Fonctions auxiliaires pour calculer AP, P@K, R@K
def calculate_average_precision(is_relevant):
    precisions = [sum(is_relevant[:i+1]) / (i+1) for i in range(len(is_relevant)) if is_relevant[i]]
    return sum(precisions) / len(precisions) if precisions else 0

def calculate_precision_at_k(is_relevant, k):
    return sum(is_relevant[:k]) / k if k <= len(is_relevant) else 0

def calculate_recall_at_k(is_relevant, k, total_relevant):
    return sum(is_relevant[:k]) / total_relevant if total_relevant > 0 else 0

# Fonctions auxiliaires pour calculer MRR, DCG, NDCG
def calculate_mrr(is_relevant):
    """ Calculate Mean Reciprocal Rank """
    # Trouver le rang du premier document pertinent
    try:
        first_relevant = is_relevant.index(1) + 1
        return 1 / first_relevant
    except ValueError:
        return 0

def calculate_dcg(scores):
    """ Calculate Discounted Cumulative Gain """
    return np.sum([rel / np.log2(idx + 2) for idx, rel in enumerate(scores)])

def calculate_ndcg(is_relevant, ideal_relevant):
    """ Calculate normalized Discounted Cumulative Gain """
    dcg = calculate_dcg(is_relevant)
    idcg = calculate_dcg(ideal_relevant)
    return dcg / idcg if idcg > 0 else 0

# Exemple d'utilisation
MAP, mean_p_at_5, mean_recall_at_5, mean_p_at_10, mean_recall_at_10, mean_recall_at_20, mean_mrr, mean_ndcg = evaluate_performance(data_dict, doc_passages)
print(f"MAP: {MAP}, P@5: {mean_p_at_5}, Recall@5: {mean_recall_at_5}, P@10: {mean_p_at_10}, Recall@10: {mean_recall_at_10}, Recall@20: {mean_recall_at_20}, MRR: {mean_mrr}, nDCG: {mean_ndcg}")


MAP: 0.5504068349028322, P@5: 0.2004807692307702, Recall@5: 0.6718349358974358, P@10: 0.11682692307692367, Recall@10: 0.7808493589743589, Recall@20: 0.8850961538461539, MRR: 0.6446488504589841, nDCG: 0.6804211017444378


Cool, on généralise sur plusieurs modèles désormais

In [15]:
from math import log, sqrt, exp

def rank_passages(query, passages, model='BM25'):
    # tokenizer la requête
    tokens = word_tokenize(query)
    # Suppression des mots vides et nettoyage des caractères non alphabétiques
    tokens = [word.lower() for word in tokens if word.lower() not in stop_words and word.isalpha()]
    # Lemmatisation
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    tokenized_query = tokens

    # Choisir le modèle de scoring
    if model == 'BM25':
        bm25 = BM25Okapi(passages)
        scores = bm25.get_scores(tokenized_query)
    elif model == 'WC':
        scores = wc_scores(passages, tokenized_query)
    elif model == 'WC.IDF':
        scores = wc_idf_scores(passages, tokenized_query)
    elif model == 'VSM':
        scores = vsm_scores(passages, tokenized_query)
    elif model == 'QL':
        scores = ql_scores(passages, tokenized_query)
    else:
        raise ValueError("Unsupported model. Choose 'BM25', 'WC', 'WC.IDF', 'VSM', or 'QL'.")

    # Trier les passages par score en ordre décroissant
    ranked_passages = np.argsort(scores)[::-1]
    
    return ranked_passages, scores

def wc_scores(passages, tokenized_query):
    scores = []
    for passage in passages:
        score = sum(passage.count(word) for word in tokenized_query)
        scores.append(score)
    return scores

def wc_idf_scores(passages, tokenized_query):
    scores = []
    num_documents = len(passages)
    idf_scores = {}
    for word in tokenized_query:
        doc_containing_word = sum(1 for passage in passages if word in passage)
        idf_scores[word] = log(num_documents / (1 + doc_containing_word))  # Using log base e
    
    for passage in passages:
        score = sum(passage.count(word) * idf_scores[word] for word in tokenized_query)
        scores.append(score)
    return scores

def vsm_scores(passages, tokenized_query):
    # Compute IDF for each term in the query
    num_documents = len(passages)
    idf = {word: log(num_documents / (1 + sum(1 for p in passages if word in p))) for word in tokenized_query}
    
    # Calculate TF-IDF for each passage
    tfidf_passages = []
    for passage in passages:
        tf = Counter(passage)
        passage_length = len(passage)
        tfidf = {term: (tf[term]/passage_length) * idf.get(term, 0) for term in tokenized_query}
        tfidf_passages.append(tfidf)
    
    # Calculate cosine similarity for each passage
    scores = []
    query_vec = {term: idf.get(term, 0) for term in tokenized_query}
    query_norm = sqrt(sum(val**2 for val in query_vec.values()))
    
    for tfidf in tfidf_passages:
        dot_product = sum(query_vec[term] * tfidf.get(term, 0) for term in tokenized_query)
        passage_norm = sqrt(sum(val**2 for val in tfidf.values()))
        if passage_norm == 0 or query_norm == 0:
            scores.append(0)
        else:
            scores.append(dot_product / (query_norm * passage_norm))
    return scores

def ql_scores_mercer(passages, tokenized_query): # Q-likelihood avec smoothing de Jelinek-Mercer
    lambda_param = 0.7
    corpus = [word for passage in passages for word in passage]
    corpus_freq = Counter(corpus)
    total_words = sum(corpus_freq.values())
    
    scores = []
    for passage in passages:
        passage_freq = Counter(passage)
        passage_length = len(passage)
        log_score = 0  # Use logarithmic score to prevent underflow
        for term in tokenized_query:
            # Calculate term probabilities with smoothing
            term_corpus_prob = corpus_freq[term] / total_words
            term_passage_prob = passage_freq[term] / passage_length if passage_length > 0 else 0
            term_prob = lambda_param * term_passage_prob + (1 - lambda_param) * term_corpus_prob
            
            # Use logarithm of probability to avoid underflow
            if term_prob > 0:
                log_score += log(term_prob)
            else:
                # Handle zero probability by assigning a low probability that avoids log(0)
                log_score += log(1e-10)

        # Convert log score back to normal score for output
        scores.append(exp(log_score))
    return scores


def ql_scores(passages, tokenized_query, mu=2000): # Query Likelihood avec smoothing de Dirichlet, comme sur le papier
    # Flatten the corpus to calculate term probabilities in the collection
    corpus = [word for passage in passages for word in passage]
    corpus_freq = Counter(corpus)
    total_corpus_words = sum(corpus_freq.values())

    scores = []
    for passage in passages:
        passage_freq = Counter(passage)
        passage_length = len(passage)
        log_score = 0  # Using log to handle product of probabilities

        for term in tokenized_query:
            term_corpus_prob = corpus_freq[term] / total_corpus_words
            term_passage_freq = passage_freq[term]
            
            # Calculate smoothed probability of the term
            smoothed_term_prob = (term_passage_freq + mu * term_corpus_prob) / (passage_length + mu)

            # Use log to sum probabilities to avoid underflow
            if smoothed_term_prob > 0:
                log_score += log(smoothed_term_prob)
            else:
                log_score += log(1e-10)  # Handle the case where term probability is zero

        scores.append(exp(log_score))  # Convert log score back to normal score for ranking

    return scores




In [16]:
def evaluate_performance(data_dict, wiki_docs, model='WC.IDF'):
    all_ap = []
    all_p_at_5 = []
    all_recall_at_5 = []
    all_p_at_10 = []
    all_recall_at_10 = []
    all_recall_at_20 = []
    all_mrr = []
    all_ndcg = []


    # Itérer sur chaque question dans data_dict
    for q_id, q_data in data_dict.items():
        query = q_data['Question']
        doc_id = str(q_data['DocumentID'])
        relevant_passages = q_data['RelevantPassages']
        
        # Obtenir les passages du document
        # passages = wiki_docs[doc_id]
        passages = fetch_passages(doc_id, wiki_docs)

        
        # Classement des passages pour la requête
        ranked_passages_index, scores = rank_passages(query, passages, model)
        
        # Identifier les passages pertinents
        is_relevant = [0] * len(passages)
        for idx, ranked_idx in enumerate(ranked_passages_index):
            for rel_id in relevant_passages:
                if calculate_bigram_overlap(passages[ranked_idx], doc_passages[doc_id][str(rel_id)]):
                    is_relevant[idx] = 1
                    break

        # Calculer les métriques pour cette requête
        ap = calculate_average_precision(is_relevant)
        p_at_5 = calculate_precision_at_k(is_relevant, 5)
        recall_at_5 = calculate_recall_at_k(is_relevant, 5, len(relevant_passages))
        p_at_10 = calculate_precision_at_k(is_relevant, 10)
        recall_at_10 = calculate_recall_at_k(is_relevant, 10, len(relevant_passages))
        recall_at_20 = calculate_recall_at_k(is_relevant, 20, len(relevant_passages))

        all_ap.append(ap)
        all_p_at_5.append(p_at_5)
        all_recall_at_5.append(recall_at_5)
        all_p_at_10.append(p_at_10)
        all_recall_at_10.append(recall_at_10)
        all_recall_at_20.append(recall_at_20)

        # Calcul des métriques MRR et nDCG
        all_mrr.append(calculate_mrr(is_relevant))

        ideal_relevant = sorted(is_relevant, reverse=True)  # Idéal pour nDCG
        all_ndcg.append(calculate_ndcg(is_relevant, ideal_relevant))

    # Calculer les moyennes pour toutes les requêtes
    MAP = sum(all_ap) / len(all_ap)
    mean_p_at_5 = sum(all_p_at_5) / len(all_p_at_5)
    mean_recall_at_5 = sum(all_recall_at_5) / len(all_recall_at_5)
    mean_p_at_10 = sum(all_p_at_10) / len(all_p_at_10)
    mean_recall_at_10 = sum(all_recall_at_10) / len(all_recall_at_10)
    mean_recall_at_20 = sum(all_recall_at_20) / len(all_recall_at_20)
    mean_mrr = np.mean(all_mrr)
    mean_ndcg = np.mean(all_ndcg)

    return MAP, mean_p_at_5, mean_recall_at_5, mean_p_at_10, mean_recall_at_10, mean_recall_at_20, mean_mrr, mean_ndcg

MODEL_LIST = ['WC', 'WC.IDF','VSM', 'QL', 'BM25']

for model in MODEL_LIST:
    MAP, mean_p_at_5, mean_recall_at_5, mean_p_at_10, mean_recall_at_10, mean_recall_at_20, mean_mrr, mean_ndcg = evaluate_performance(data_dict, doc_passages, model)
    print(f"Model : {model}, MAP: {MAP}, P@5: {mean_p_at_5}, Recall@5: {mean_recall_at_5}, P@10: {mean_p_at_10}, Recall@10: {mean_recall_at_10}, Recall@20: {mean_recall_at_20}, MRR: {mean_mrr}, nDCG: {mean_ndcg}")


Model : WC, MAP: 0.3438628545425129, P@5: 0.1413461538461542, Recall@5: 0.48585737179487193, P@10: 0.09230769230769266, Recall@10: 0.6405048076923077, Recall@20: 0.8021233974358973, MRR: 0.39557709388068163, nDCG: 0.5094462285195396
Model : WC.IDF, MAP: 0.5466203767386509, P@5: 0.2004807692307702, Recall@5: 0.6666266025641027, P@10: 0.11802884615384678, Recall@10: 0.7863782051282052, Recall@20: 0.896113782051282, MRR: 0.6319142882340731, nDCG: 0.6765482331705374
Model : VSM, MAP: 0.5050833246245656, P@5: 0.19038461538461632, Recall@5: 0.6338541666666669, P@10: 0.1146634615384621, Recall@10: 0.7568509615384615, Recall@20: 0.8858974358974359, MRR: 0.5921356923353754, nDCG: 0.6452622690369151
Model : QL, MAP: 0.5336536811893733, P@5: 0.200000000000001, Recall@5: 0.6707131410256412, P@10: 0.11850961538461599, Recall@10: 0.7903846153846155, Recall@20: 0.8933092948717949, MRR: 0.6293490217250438, nDCG: 0.6688587156960991
Model : BM25, MAP: 0.5504068349028322, P@5: 0.2004807692307702, Recall@